<a href="https://colab.research.google.com/github/Nourhan-Mahmoud/Neural-Network-Tasks/blob/main/Final_Project/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras import datasets, layers, models
from keras.callbacks import TensorBoard
import cv2
from tensorflow.keras import callbacks


In [52]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [53]:
# ! kaggle competitions download nn23-sports-image-classification && unzip nn23-sports-image-classification.zip

In [54]:
train_files = os.listdir('Train')

In [55]:
train_labels = [item.split('_')[0] for item in train_files]

In [56]:
class_names=['Basketball',
'Football',
'Rowing',
'Swimming',
'Tennis',
'Yoga']

In [57]:
try:
  os.mkdir('data')
  for i in class_names:
    os.mkdir(f'data/{i}')
except:  pass

In [58]:
import shutil

In [59]:
for i in train_files:
  #move i to folder
  try: shutil.move(f"Train/{i}",f"data/{ i.split('_')[0]}")
  except: pass


In [60]:
from keras_preprocessing.image import ImageDataGenerator

trainDataGenerator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
testDataGenerator = ImageDataGenerator(rescale=1./255)

In [61]:
train_d = trainDataGenerator.flow_from_directory(
     "/content/data/",
      classes=class_names,
      color_mode="rgb",
      batch_size=32,
      target_size=(64,64),
      class_mode="categorical",
      subset="training")

valid_d = trainDataGenerator.flow_from_directory(
    "/content/data/",
    classes=class_names,
    color_mode="rgb",
    batch_size=32,
    target_size=(64,64),
    class_mode="categorical",
    subset="validation"
)

Found 1346 images belonging to 6 classes.
Found 335 images belonging to 6 classes.


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
model = Sequential()
model.add(Conv2D(128,(3,3), input_shape=(64,64,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(rate=0.2))
model.add(Dense(6, activation="softmax"))
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
callback_list=[
    callbacks.EarlyStopping(monitor="val_accuracy",patience=10,restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_accuracy",patience=3)
]
history = model.fit(train_d,validation_data=valid_d,epochs=50,verbose=1,callbacks=callback_list)

In [ ]:
import keras
testlst=[]
sub={'image_name':[],'label':[]}
for i in os.listdir('Test'):
   img = keras.utils.load_img(f'/content/Test/{i}',target_size=(64, 64))
   input_arr = tf.keras.preprocessing.image.img_to_array(img)
   input_arr = np.array([input_arr])  
   input_arr = input_arr.astype('float32') / 255.
   res=model.predict(input_arr)
   sub['image_name'].append(i)
   sub['label'].append(res.argmax())


In [65]:
dd=pd.DataFrame.from_dict(sub)

In [70]:
dd[dd['image_name']=='1.jpg']

,image_name,label
443,1.jpg,4


In [72]:
dd.to_csv('sub.csv',index=False)
